In [ ]:
import cv2
import numpy as np
from plot import FacemarksPlot, EmotionPlot
from src.dlib_wrapper import *
from src.utils import *
detector = dlib_face_frontal_detector()
predictor2d = dlib_face_landmarks_predictor()
predictor3d = get_fm_depth_predictor()
landmarks_transform = get_landmarks_transform()
face_images_transform = get_face_images_transform()
num_classes = 5
predictor_fer_fc = get_fc_fer_predictor(num_classes=num_classes)
predictor_fer_cnn = get_cnn_fer_predictor(num_classes=num_classes)
predictor_fer_cascade = get_cascade_fer_predictor(num_classes=num_classes)
get_features = lambda image: get_face_features(image, detector, predictor2d, predictor3d, landmarks_transform)

In [ ]:
fplot = FacemarksPlot()
# eplot = EmotionPlot()
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret or cv2.waitKey(1) == 27: break

        ok, rectangles, landmarks, facemarks = get_features(frame)
        if ok:
            biometrics = get_face_biometrics(facemarks)
            faces = get_face_images(frame, rectangles, face_images_transform)
            # expressions = get_face_expression(biometrics, predictor_fer_fc)
            # expressions = get_face_expression(faces, predictor_fer_cnn)
            expressions = get_face_expression((faces, biometrics), predictor_fer_cascade)
            print(expressions, '\n')
            for i, rectangle in enumerate(rectangles):
                x, y, w, h = dlib_rectangle_to_numpy_box(frame.shape, rectangle)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 1)
                out = expressions['output'][i]
                text = '{}: {:.3f}'.format(out, expressions[out][i])
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        fplot.show2d(frame, ok, landmarks, mesh=False)
        fplot.show3d(ok, facemarks)
        # eplot.show_bar(ok, expressions)

except KeyboardInterrupt: pass

cap.release()
cv2.destroyAllWindows()